In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Legal Entities - Creating Legal Entities and Relationships

Attributes
----------
legal entity
relationship
custodian
portfolios
"""

toggle_code("Toggle Docstring")

# Creating Legal Entities and Relationships

In this notebook, we demonstrate how you can set up a legal entity and link it with various assets using relationships.

## Table of Contents:
* [1. Create Entity Properties](#1.-Create-Entity-Properties)
* [2. Create a Legal Entity](#2.-Create-a-Legal-Entity)
* [3. Create a Portfolio](#3.-Create-a-Portfolio)
* [4. Create and Set up Relationships](#4.-Create-and-Set-up-Relationships)

In [1]:
# Import generic non-LUSID packages
import os
import pandas as pd
import numpy as np
from datetime import datetime
import json
import pytz
import time
from IPython.core.display import HTML

# Import key modules from the LUSID package
import lusid as lu
import lusid.models as lm

# Import key functions from Lusid-Python-Tools and other packages
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.transaction_type_upload import upsert_transaction_type_alias
from lusidtools.lpt.lpt import to_date
from lusidjam import RefreshingToken


# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

# display(HTML("<style>.container { width:90% !important; }</style>"))

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Authenticate our user and create our API client
api_factory = lu.utilities.ApiClientFactory(
    token=RefreshingToken(), api_secrets_filename=secrets_path
)

print("LUSID Environment Initialised")
print(
    "LUSID API Version :",
    api_factory.build(lu.api.ApplicationMetadataApi).get_lusid_versions().build_version,
)

LUSID Environment Initialised
LUSID API Version : 0.6.9360.0


In [2]:
scope = "legalEntitiesNotebook"

In [3]:
relationship_definitions_api = api_factory.build(lu.RelationshipDefinitionsApi)
relationships_api = api_factory.build(lu.RelationshipsApi)
property_definitions_api = api_factory.build(lu.api.PropertyDefinitionsApi)
transaction_portfolios_api = api_factory.build(lu.api.TransactionPortfoliosApi)
portfolios_api = api_factory.build(lu.api.PortfoliosApi)
legal_entities_api = api_factory.build(lu.api.LegalEntitiesApi)

A legal entity is in essence a collection of properties. One property to identify it by and then a group of properties that determine it's characteristics. We must first create a property identifier and then create some properties so that we can set up the legal entity.

## 1. Create Entity Properties

### Create identifier property

There are two things to look out for here, we must set the domain to "LegalEntity" and the constraint style to "Identifier" so that lusid knows this is meant to be an identifier property for a legal entity.

In [5]:
def create_property_identifier(code, display_name):
    try:
        property_definitions_api.create_property_definition(
            create_property_definition_request=lm.CreatePropertyDefinitionRequest(
                domain="LegalEntity",
                scope=scope,
                code=code,
                value_required=None,
                display_name=display_name,
                data_type_id=lm.ResourceId(scope="system", code="string"),
                life_time="Perpetual",
                constraint_style="Identifier"
            )
        )
    except:
       print("Property already exists")

Here we create a property identifier called Custodian, so that we can make legal entities out of our custodians later on.

In [6]:
create_property_identifier("Custodian", "Custodian")

Once we have created the identifier 'Custodian' we can now create some properties to fill in later that will be part of the legal entity.

In [7]:
def create_property(domain, dtype, code, display_name):
    try:
        property_definitions_api.create_property_definition(
            create_property_definition_request=lm.CreatePropertyDefinitionRequest(
                domain=domain,
                scope=scope,
                code=code,
                value_required=None,
                display_name=display_name,
                data_type_id=lm.ResourceId(scope="system", code=dtype),
                life_time="Perpetual",
            )
        )
    except:
       print("Property already exists")

Our legal entities will have three properties in this example; a code to refer to the custodian internally, the name of the custodian and the country the custodian operates in.

In [17]:
create_property("LegalEntity", "string", "custodian_code", "Custodian Code")
create_property("LegalEntity", "string", "custodian_name", "Custodian Name")
create_property("LegalEntity", "string", "country", "Country")

Property already exists


## 2. Create a Legal Entity

In [18]:
def create_legal_entity(identifier_property, custodian_code, custodian_name, country):

    legal_entity_request= lm.UpsertLegalEntityRequest(
    identifiers = {f"LegalEntity/{scope}/{identifier_property}": lm.PerpetualProperty(
        key=f"LegalEntity/{scope}/{identifier_property}",
        value=lm.PropertyValue(
            label_value=custodian_code
        )
    )},
    properties = {
        f"LegalEntity/{scope}/custodian_code":
        lm.PerpetualProperty(
            key=f"LegalEntity/{scope}/custodian_code",
            value=lm.PropertyValue(
                label_value=custodian_code
            )
        ),
        f"LegalEntity/{scope}/custodian_name":
        lm.PerpetualProperty(
            key=f"LegalEntity/{scope}/custodian_name",
            value=lm.PropertyValue(
                label_value=custodian_name
            )
        ),
        f"LegalEntity/{scope}/country":
        lm.PerpetualProperty(
            key=f"LegalEntity/{scope}/country",
            value=lm.PropertyValue(
                label_value=country
            )
        ),
    },
    display_name = identifier_property +" "+ custodian_name,
    description = None,
    counterparty_risk_information = None,
    )

    legal_entities_api.upsert_legal_entity(legal_entity_request)
    print("created legal entity "+ custodian_code)

In [19]:
create_legal_entity("Custodian", "CITI", "Citi", "US")
create_legal_entity("Custodian", "SSTREET", "State Street", "US")
create_legal_entity("Custodian", "BNYM", "Bank of New York Mellon", "US")

created legal entity CITI
created legal entity SSTREET
created legal entity BNYM


## 3. Create a Portfolio

We now have our legal entities ready to be linked to some portfolios or other entities within LUSID. Let's create a portfolio and set up a relationship with one of our new custodian legal entities.

In [11]:
try:
    transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name="Legal Entities Notebook Test Portfolio",
            code="legalEntitiesNotebookPortfolio",
            base_currency="USD",
            created="2022-01-01",
            sub_holding_keys=[],
        ),
    )

except lu.ApiException as e:
    print(json.loads(e.body)["title"])

## 4. Create and Set up Relationships

Now that we have the legal entity and the portfolio, let's describe to LUSID what a relationship is between a portfolio and a custodian.

In [12]:
def create_relationship_definition(code, display_name, outward_description, inward_description):
    try:
        relationship_definitions_api.create_relationship_definition(
            create_relationship_definition_request=lu.CreateRelationshipDefinitionRequest(
                scope=scope,
                code=code,
                source_entity_type="LegalEntity", 
                target_entity_type="Portfolio", 
                display_name=display_name, 
                outward_description=outward_description, 
                inward_description=inward_description, 
                life_time="TimeVariant"
            )
        )
        print(f"created relation {scope}/{code}")
    except lu.ApiException as e:
        body = json.loads(e.body)
        if body["code"] != 667:  # RelationDefinitionAlreadyExists
            print(body)
        else:
            print(f"relation {scope}/{code} already exists")

In [13]:
create_relationship_definition("custodian", "Custodian link to portfolio", "Custodian of", "Portfolio in custody by")

created relation legalEntitiesNotebook/custodian


Now that we have defined a relationship to a custodian, we can now set up one between our custodian BNYM and the portfolio.

In [20]:
def create_custodian_relationship(relation, id_code, id_value, to_portfolio):
    relationships_api.create_relationship(
        
        # the scope/code of the RelationDefinition to be created
        scope=scope,
        code=relation,
        
        create_relationship_request=lm.CreateRelationshipRequest(
            source_entity_id={
                # the fields the uniquely identify the source entity
                "idTypeScope": scope,
                "idTypeCode": id_code,
                "code": id_value
            }, 
            target_entity_id={
                # the fields the uniquely identify the target entity
                "scope": scope,
                "code": to_portfolio
            },
            effective_from='2022-01-01T00:00:00+00:00'
            ),
    )

In [21]:
create_custodian_relationship("custodian", "Custodian", "BNYM", "legalEntitiesNotebookPortfolio")

We now have a custodian as a legal entity called BNYM, we have a portfolio and we have a relationship between the portfolio and the custodian legal entity. We can display this by calling the get_portfolio_relationships() endpoint of the portfolios API.

In [24]:
relationships = portfolios_api.get_portfolio_relationships(
    scope=scope,
    code="legalEntitiesNotebookPortfolio"
)

relationships_table = lusid_response_to_data_frame(relationships)
relationships_table

,relationship_definition_id.scope,relationship_definition_id.code,related_entity.entity_type,related_entity.entity_id.idTypeScope,related_entity.entity_id.idTypeCode,related_entity.entity_id.code,related_entity.display_name,related_entity.properties,traversal_direction,traversal_description,effective_from
0,legalEntitiesNotebook,custodian,LegalEntity,legalEntitiesNotebook,Custodian,BNYM,Custodian Bank of New York Mellon,{},In,Portfolio in custody by,2022-01-01 00:00:00+00:00


We can make the relationship even easier to see if we just call the traversal description and the entity name of the table above. We can then see that our portfolio "legalEntitiesNotebookPortfolio" is a portoflio in custody by Bank of New York Mellon.

In [25]:
relationships_table[['traversal_description', 'related_entity.display_name']]

,traversal_description,related_entity.display_name
0,Portfolio in custody by,Custodian Bank of New York Mellon
